In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import DBSCAN
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge, Lasso
from sklearn import preprocessing
import xgboost as xgb

In [2]:
# Step 0: Load the proper csvs

xtrain = pd.read_csv("X_train.csv")
xtrain = xtrain.iloc[:, 1:]
xtest = pd.read_csv("X_test.csv")
xtest = xtest.iloc[:, 1:]
ytrain = pd.read_csv("Y_train.csv")

In [3]:
# Step 1: Treat missing values
# Treat missing values as column medians. Important (apparently) to use the medians from the training set in the test set

xtrain = xtrain.fillna(xtrain.median())
xtest = xtest.fillna(xtrain.median())

In [4]:
# Step 2: Scale the training and test data

scaler = preprocessing.StandardScaler()

xtrain_scaled = scaler.fit_transform(xtrain)
xtrain = pd.DataFrame(xtrain_scaled, columns = xtrain.columns)
xtest_scaled = scaler.fit_transform(xtest)
xtest = pd.DataFrame(xtest_scaled, columns = xtest.columns)

In [5]:
# Step 3: Remove unnecessary id column from ytrain that just ***** things up

ytrain1 = ytrain.loc[:, "y"]
ytrain2 = pd.DataFrame(data = ytrain1.values, columns= ['y'])
ytrain = ytrain2

In [6]:
# Remove features with zero weight from Lasso

clf = Lasso(alpha=0.3)
clf.fit(xtrain, ytrain)

features = (clf.coef_ != 0)

xtrain = xtrain.loc[:, features]
xtest = xtest.loc[:, features]

In [7]:



# Do weird Andreas PCA thing

from sklearn import decomposition
import matplotlib.pyplot as plt
pca = decomposition.PCA(n_components=2)
principal_components = pca.fit_transform(xtrain)
xpca = pd.DataFrame(data = principal_components, columns = ['PC1', 'PC2'])

outliers = (xpca['PC1'] <= 9) & (xpca['PC1'] >= -8) & (xpca['PC2'] <= 7) & (xpca['PC2'] >= -7)
print(np.count_nonzero(outliers))
xtrain = xtrain[outliers]
ytrain = ytrain[outliers]

'''

# Outlier detection with local outlier factor
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(n_neighbors=700, contamination=0.08)
outliers = clf.fit_predict(xtrain)

# Remove outliers from xtrain and ytrain


outliers = outliers == 1
print(np.count_nonzero(outliers))
xtrain = xtrain[outliers]
ytrain = ytrain[outliers]

'''

1155


'\n\n# Outlier detection with local outlier factor\nfrom sklearn.neighbors import LocalOutlierFactor\n\nclf = LocalOutlierFactor(n_neighbors=700, contamination=0.08)\noutliers = clf.fit_predict(xtrain)\n\n# Remove outliers from xtrain and ytrain\n\n\noutliers = outliers == 1\nprint(np.count_nonzero(outliers))\nxtrain = xtrain[outliers]\nytrain = ytrain[outliers]\n\n'

# Run Bayesian Optimization on XGBoost parameters

In [8]:
# k-fold cross validation evaluation of xgboost model

from numpy import loadtxt
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold


clf = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)



cv_means = []
cv_stds = []
for i in np.arange(10):
    scores = cross_val_score(estimator = clf,
                                 X = xtrain,
                                 y = ytrain,
                                 scoring = 'r2',
                                 cv = KFold(n_splits=5, shuffle = True))
    cv_means.append(np.mean(scores))
    cv_stds.append(np.std(scores))

print(cv_means)
print(cv_stds)
print(np.mean(cv_means))
print(np.mean(cv_stds))

[0.6491839050176116, 0.6471371854589318, 0.6391528990274764, 0.6340039064060577, 0.6415884405202968, 0.6494298605082397, 0.65346915707234, 0.6405600475093319, 0.6494465034886889, 0.6423864066374126]
[0.026554213685352098, 0.03971028940539932, 0.02038010009558867, 0.026801380582643773, 0.03162171239701299, 0.025048251358722844, 0.023367925631136332, 0.011882191344254816, 0.02484607821746095, 0.05788755333923325]
0.6446358311646387
0.028809969605680507


In [8]:
from bayes_opt import BayesianOptimization
from numpy import loadtxt
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold

# Specify the function
def xgbfun(max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, objective='reg:squarederror', booster='gbtree', tree_method='auto', n_jobs=1, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, missing=None, num_parallel_tree=1, importance_type='gain'):
    model = xgb.XGBRegressor(max_depth=int(max_depth), learning_rate=learning_rate, n_estimators=int(n_estimators), verbosity=1, objective='reg:squarederror', booster='gbtree', tree_method='auto', n_jobs=1, 
                             gamma=gamma, min_child_weight=int(min_child_weight), max_delta_step=int(max_delta_step), subsample=subsample, 
                             colsample_bytree=colsample_bytree, colsample_bylevel=colsample_bylevel, colsample_bynode=colsample_bynode, 
                             reg_alpha=reg_alpha, reg_lambda=reg_lambda, scale_pos_weight=scale_pos_weight, base_score=0.5, random_state=42, 
                             missing=None, num_parallel_tree=1, importance_type='gain')
    
    scores = cross_val_score(estimator = model,
                                 X = xtrain,
                                 y = ytrain,
                                 scoring = 'r2',
                                 cv = 10)
    return np.mean(scores)



# specify parameters and distributions to sample from
param_dist = {"max_depth": (2, 5),
              "learning_rate": (0.1, 0.9),
              "n_estimators": (1, 200),
              "gamma": (0.001, 2),
              "min_child_weight": (0.01, 3),
              "max_delta_step": (0.01, 3),
              "subsample": (0.01, 1),
              "colsample_bytree": (0.01, 1),
              "colsample_bylevel": (0.01, 1), 
              "colsample_bynode": (0.01, 1),
              "reg_alpha": (0, 1),
              "reg_lambda": (0, 1),
              "scale_pos_weight": (0.001, 1)}

optimizer = BayesianOptimization(
    f=xgbfun,
    pbounds=param_dist,
    verbose=2,
    random_state=1,
)

probe_params = { "max_depth":3, 
                "learning_rate":0.1, 
                "n_estimators":100, 
                "gamma":0, 
                "min_child_weight":1, 
                "max_delta_step":0, 
                "subsample":1, 
                "colsample_bytree":1, 
                "colsample_bylevel":1, 
                "colsample_bynode":1, 
                "reg_alpha":0, 
                "reg_lambda":1, 
                "scale_pos_weight":1 }
optimizer.probe(
    params=probe_params,
    lazy=True
)

optimizer.maximize(
    init_points=2,
    n_iter=200,
)

print(optimizer.max)

|   iter    |  target   | colsam... | colsam... | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6417   |  1.0      |  1.0      |  1.0      |  0.0      |  0.1      |  0.0      |  3.0      |  1.0      |  100.0    |  0.0      |  1.0      |  1.0      |  1.0      |
|  2        |  0.4355   |  0.4229   |  0.7231   |  0.01011  |  0.6054   |  0.2174   |  0.2861   |  2.559    |  1.043    |  79.96    |  0.5388   |  0.4192   |  0.6855   |  0.2124   |
|  3        |  0.4923   |  0.8793   |  0.03711  |  0.6738   |  0.8352   |  0.547    |  0.4298   |  2.594    |  2.404    |  193.7    |  0.3134   |  0.6923   |  0.8765   |  0.8957   |
|  4        | -49.47    |  1.0      |  1.0      |  1.0      |  0.001    |  0.9      |  3.0

|  27       |  0.5576   |  0.9428   |  0.9427   |  0.9367   |  0.1259   |  0.7563   |  2.454    |  4.759    |  2.582    |  96.15    |  0.03485  |  0.9652   |  0.9372   |  0.9409   |
|  28       |  0.6337   |  0.918    |  0.7615   |  1.0      |  0.001    |  0.1811   |  0.468    |  3.429    |  1.067    |  99.79    |  0.5632   |  1.0      |  1.0      |  1.0      |
|  29       |  0.5603   |  0.5344   |  0.5081   |  1.0      |  0.001    |  0.3589   |  0.4176   |  3.594    |  1.095    |  99.31    |  0.0      |  1.0      |  1.0      |  0.8619   |
|  30       |  0.5057   |  1.0      |  1.0      |  0.47     |  0.1172   |  0.4723   |  0.3869   |  3.555    |  1.166    |  99.27    |  0.0      |  1.0      |  1.0      |  0.7541   |
|  31       |  0.6465   |  0.6647   |  1.0      |  1.0      |  0.001    |  0.1      |  0.8349   |  3.348    |  1.108    |  99.87    |  0.02086  |  0.7965   |  1.0      |  1.0      |
|  32       |  0.6451   |  1.0      |  0.6074   |  0.8424   |  0.09983  |  0.1      |  0.4

|  55       |  0.6368   |  0.8672   |  0.7769   |  0.8996   |  0.4533   |  0.1722   |  0.5021   |  3.229    |  1.304    |  99.28    |  0.0      |  0.6159   |  0.4982   |  1.0      |
|  56       |  0.6387   |  0.891    |  0.745    |  0.5189   |  0.09894  |  0.1      |  0.2125   |  3.324    |  1.006    |  99.42    |  0.4426   |  0.6154   |  0.3771   |  1.0      |
|  57       |  0.6398   |  0.9898   |  0.4248   |  0.5766   |  0.2851   |  0.1      |  0.3794   |  3.319    |  0.8719   |  99.4     |  0.0      |  0.6066   |  0.7312   |  0.5363   |
|  58       |  0.6385   |  0.905    |  0.5882   |  1.0      |  0.2234   |  0.1      |  0.3609   |  3.257    |  1.087    |  99.5     |  0.5029   |  0.849    |  0.2892   |  0.5262   |
|  59       |  0.641    |  0.8666   |  0.3832   |  1.0      |  0.148    |  0.1      |  0.359    |  3.56     |  0.7303   |  99.35    |  0.04358  |  0.7213   |  0.379    |  1.0      |
|  60       |  0.6511   |  0.9909   |  0.9214   |  0.8687   |  0.2192   |  0.1      |  0.1

|  83       |  0.5332   |  0.8675   |  0.8748   |  0.8604   |  0.2501   |  0.7086   |  2.276    |  4.751    |  2.39     |  95.92    |  0.1438   |  0.8551   |  0.8754   |  0.8633   |
|  84       |  0.6281   |  0.7271   |  0.7088   |  0.3702   |  0.07362  |  0.1696   |  0.5189   |  3.159    |  0.5601   |  99.78    |  0.2613   |  0.8084   |  0.6267   |  0.7858   |
|  85       |  0.6253   |  0.9044   |  0.8474   |  0.9539   |  0.07689  |  0.2068   |  0.3858   |  3.384    |  1.073    |  99.51    |  0.3406   |  0.7128   |  0.4342   |  0.8895   |
|  86       |  0.5031   |  0.9284   |  0.9326   |  0.9259   |  0.1325   |  0.7962   |  2.609    |  4.865    |  2.668    |  95.77    |  0.08398  |  0.9154   |  0.9343   |  0.9265   |
|  87       |  0.6307   |  0.8251   |  0.7884   |  0.8003   |  0.2524   |  0.2374   |  0.4084   |  3.308    |  1.019    |  99.25    |  0.05684  |  0.8903   |  0.7983   |  0.7868   |
|  88       |  0.623    |  0.7151   |  0.8487   |  0.6546   |  0.02022  |  0.1946   |  0.5

|  111      |  0.6327   |  0.4999   |  0.7141   |  0.5712   |  0.6007   |  0.1835   |  0.2178   |  3.554    |  1.074    |  100.0    |  0.4401   |  0.5939   |  0.8733   |  0.8053   |
|  112      |  0.6307   |  0.5991   |  0.6706   |  0.4785   |  0.305    |  0.1559   |  0.2352   |  3.355    |  0.8198   |  99.71    |  0.4024   |  0.7908   |  0.7525   |  0.7642   |
|  113      |  0.5957   |  0.6862   |  0.9434   |  0.4711   |  0.2908   |  0.2627   |  0.2544   |  2.997    |  0.6127   |  99.36    |  0.1591   |  0.6656   |  0.8661   |  0.4959   |
|  114      |  0.6365   |  0.7663   |  0.8354   |  0.5774   |  0.238    |  0.1905   |  0.2706   |  3.046    |  0.5096   |  99.56    |  0.437    |  0.8901   |  0.6553   |  0.8245   |
|  115      |  0.6439   |  0.677    |  0.8048   |  0.6994   |  0.2286   |  0.1      |  0.318    |  3.032    |  0.8854   |  99.46    |  0.1512   |  0.7074   |  0.9858   |  0.8537   |
|  116      |  0.6256   |  0.8819   |  0.833    |  0.7093   |  0.1811   |  0.1647   |  0.2

|  139      |  0.6393   |  0.5092   |  0.9421   |  1.0      |  0.001    |  0.1668   |  0.6477   |  3.29     |  1.062    |  99.43    |  0.00103  |  0.8201   |  0.9603   |  0.8464   |
|  140      |  0.6467   |  0.9793   |  0.9197   |  0.9655   |  0.2041   |  0.1      |  0.2755   |  3.318    |  0.9404   |  99.32    |  0.4522   |  0.9982   |  0.7594   |  0.8826   |
|  141      |  0.6259   |  0.9736   |  0.6947   |  0.9365   |  0.001    |  0.2076   |  0.4301   |  3.34     |  1.214    |  99.89    |  0.0      |  0.5949   |  1.0      |  0.9555   |
|  142      |  0.6313   |  0.3992   |  0.6972   |  0.229    |  0.1385   |  0.1099   |  0.5905   |  3.289    |  0.8378   |  99.79    |  0.1574   |  0.7693   |  0.9048   |  0.7695   |
|  143      |  0.6421   |  1.0      |  0.6756   |  0.7844   |  0.1653   |  0.1      |  0.2361   |  3.392    |  0.9025   |  99.75    |  0.07529  |  0.5212   |  0.3891   |  0.894    |
|  144      |  0.618    |  0.8475   |  0.9357   |  0.3551   |  0.5226   |  0.1929   |  0.1

|  167      |  0.6475   |  1.0      |  0.6643   |  1.0      |  0.001    |  0.1      |  0.2575   |  3.486    |  0.6684   |  99.58    |  0.5351   |  0.7802   |  0.4438   |  0.7855   |
|  168      |  0.6348   |  1.0      |  0.6097   |  0.6293   |  0.001    |  0.1      |  0.511    |  3.382    |  0.6952   |  99.25    |  0.4092   |  0.936    |  0.5566   |  1.0      |
|  169      |  0.5542   |  0.7044   |  0.08443  |  0.5838   |  0.06124  |  0.6282   |  2.449    |  2.666    |  0.9502   |  191.1    |  0.6598   |  0.5541   |  0.5826   |  0.9287   |
|  170      |  0.6352   |  0.6409   |  0.7946   |  0.9533   |  0.2403   |  0.1825   |  0.5018   |  3.166    |  0.8396   |  99.45    |  0.2289   |  0.3518   |  0.2492   |  0.8484   |
|  171      |  0.6479   |  1.0      |  0.7698   |  0.9498   |  0.001    |  0.1      |  0.4246   |  3.339    |  0.9133   |  99.59    |  0.4756   |  0.436    |  0.8203   |  1.0      |
|  172      |  0.6374   |  0.7561   |  0.9519   |  0.5248   |  0.001    |  0.1      |  0.1

|  195      |  0.6167   |  0.522    |  0.8403   |  0.3478   |  0.3417   |  0.2166   |  0.4215   |  3.674    |  1.155    |  99.64    |  0.2566   |  0.6838   |  1.0      |  0.7931   |
|  196      |  0.6376   |  0.9016   |  0.6082   |  0.2867   |  0.3616   |  0.1      |  0.5543   |  3.491    |  0.3631   |  99.82    |  0.1824   |  0.8042   |  0.247    |  0.7291   |
|  197      |  0.6482   |  1.0      |  0.6918   |  1.0      |  0.001    |  0.1      |  0.4164   |  3.856    |  1.118    |  99.75    |  0.3484   |  0.6583   |  1.0      |  0.8307   |
|  198      |  0.6464   |  0.4563   |  1.0      |  0.9727   |  0.001    |  0.1      |  0.2004   |  3.19     |  1.147    |  99.48    |  0.5216   |  0.7488   |  0.3843   |  0.7098   |
|  199      |  0.6154   |  0.5655   |  0.8766   |  0.1282   |  0.3499   |  0.1      |  0.6025   |  3.264    |  0.6139   |  99.67    |  0.1675   |  0.7054   |  0.547    |  0.4527   |
|  200      |  0.6421   |  1.0      |  0.8843   |  0.595    |  0.001    |  0.1033   |  0.4

# Optimizer found the following max:

{'params': {'colsample_bylevel': 0.5353897298174576,
  'colsample_bynode': 0.9143062230315995,
  'colsample_bytree': 0.6120552891547169,
  'gamma': 0.001,
  'learning_rate': 0.1,
  'max_delta_step': 0.32938061735583135,
  'max_depth': 3.083538576010634,
  'min_child_weight': 0.7394603332174594,
  'n_estimators': 99.53230647436956,
  'reg_alpha': 0.37206475685082335,
  'reg_lambda': 0.9178582198242464,
  'scale_pos_weight': 0.9119811686410297,
  'subsample': 0.4362866767805317},
 'target': 0.659532507321262}

In [17]:
# Run XGBoost on these optimal parameters and save to csv


model = xgb.XGBRegressor(colsample_bylevel= 0.5353897298174576, 
                         colsample_bynode =0.9143062230315995, 
                         colsample_bytree =0.6120552891547169, 
                         gamma =0.001, 
                         learning_rate =0.1, 
                         max_delta_step =int(0.32938061735583135), 
                         max_depth =int(3.083538576010634), 
                         min_child_weight =int(0.7394603332174594), 
                         n_estimators =int(99.53230647436956), 
                         reg_alpha =0.37206475685082335, 
                         reg_lambda =0.9178582198242464, 
                         scale_pos_weight =0.9119811686410297, 
                         subsample =0.4362866767805317,
                         objective="reg:squarederror", 
                         random_state=42)
    
model.fit(xtrain, ytrain)

y_pred = model.predict(xtest)

index = pd.read_csv("sample.csv")
index['y'] = y_pred

index.to_csv("predictionsLassoXGB_Bopt.csv")

In [19]:
scores = cross_val_score(estimator = clf,
                             X = xtrain,
                             y = ytrain,
                             scoring = 'r2',
                             cv = 10)
print(np.mean(scores))
print(np.std(scores))

0.6417232824885015
0.04520729339953197
